# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim


gensim==4.2.0


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [2]:
import numpy as np


In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


2023-11-16 15:50:20.822664: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/tanushrinayak/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-16 15:50:25.160625: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Co

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE0Z2B3/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE0Z2B3/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [11]:
print(X_train[0])
# How many sentences are there in the training data?
print("Number of training sentences: ", len(X_train))


['this', 'was', 'an', 'absolutely', 'terrible', 'movie', "don't", 'be', 'lured', 'in', 'by', 'christopher', 'walken', 'or', 'michael', 'ironside', 'both', 'are', 'great', 'actors', 'but', 'this', 'must', 'simply', 'be', 'their', 'worst', 'role', 'in', 'history', 'even', 'their', 'great', 'acting', 'could', 'not', 'redeem', 'this', "movie's", 'ridiculous', 'storyline', 'this', 'movie', 'is', 'an', 'early', 'nineties', 'us', 'propaganda', 'piece', 'the', 'most', 'pathetic', 'scenes', 'were', 'those', 'when', 'the', 'columbian', 'rebels', 'were', 'making', 'their', 'cases', 'for', 'revolutions', 'maria', 'conchita', 'alonso', 'appeared', 'phony', 'and', 'her', 'pseudo', 'love', 'affair', 'with', 'walken', 'was', 'nothing', 'but', 'a', 'pathetic', 'emotional', 'plug', 'in', 'a', 'movie', 'that', 'was', 'devoid', 'of', 'any', 'real', 'meaning', 'i', 'am', 'disappointed', 'that', 'there', 'are', 'movies', 'like', 'this', 'ruining', "actor's", 'like', 'christopher', "walken's", 'good', 'name'

In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [5]:
# YOUR CODE HERE
# Train a word2vec model on the training set
from gensim.models import Word2Vec

word2vec_transfer = Word2Vec(X_train, window=5, min_count=1, workers=4)
word2vec_transfer.train(X_train, total_examples=len(X_train), epochs=10)


(4401543, 5827330)

Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec_transfer, X_train)
X_test_embed = embedding(word2vec_transfer, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)


☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [7]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec_transfer.wv.vector_size


assert X_train_pad.shapprinte[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)


# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [8]:
# YOUR CODE HERE
# Baseline accuracy by predicting the most frequent class in y_train

from collections import Counter

def baseline_accuracy(y_train, y_test):
    # Count the number of occurences of each class in y_train
    counts = Counter(y_train)

    # Get the most frequent class
    most_frequent_class = counts.most_common(1)[0][0]

    # Compute the accuracy of the baseline model
    accuracy = sum(y_test == most_frequent_class) / len(y_test)

    return accuracy

acc_ = baseline_accuracy(y_train, y_test)

print("Baseline accuracy: {:.2f}%".format(acc_*100))


Baseline accuracy: 49.20%


# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [12]:
# YOUR CODE HERE
# Create a RNN model with
# Masking layer and
# LTSM layer with 20 units and tanh and
# Dense layer with 10 units and
# an output layer with 1 unit and sigmoid activation

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

model = Sequential([
    Masking(mask_value=0.0, input_shape=(200, 100)),
    LSTM(20, activation='tanh'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model using rmsprop optimizer and binary_crossentropy loss
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [13]:
# YOUR CODE HERE
# Fit the model on the embedded and padded training sentences with
# early stopping patience of 5 epochs and
# batch size of 32 and
# 10 epochs

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X_train_pad,
    y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    callbacks=[early_stopping],
)


Epoch 1/10
63/63 [==============================] - 8s 68ms/step - loss: 0.6918 - accuracy: 0.5115 - val_loss: 0.6727 - val_accuracy: 0.6000
Epoch 2/10
63/63 [==============================] - 3s 50ms/step - loss: 0.6625 - accuracy: 0.5970 - val_loss: 0.6502 - val_accuracy: 0.6420
Epoch 3/10
63/63 [==============================] - 3s 54ms/step - loss: 0.6237 - accuracy: 0.6570 - val_loss: 0.6175 - val_accuracy: 0.6660
Epoch 4/10
63/63 [==============================] - 3s 54ms/step - loss: 0.5819 - accuracy: 0.7085 - val_loss: 0.6029 - val_accuracy: 0.6860
Epoch 5/10
63/63 [==============================] - 3s 51ms/step - loss: 0.5479 - accuracy: 0.7205 - val_loss: 0.6125 - val_accuracy: 0.6780
Epoch 6/10
63/63 [==============================] - 3s 51ms/step - loss: 0.5222 - accuracy: 0.7460 - val_loss: 0.5601 - val_accuracy: 0.7180
Epoch 7/10
63/63 [==============================] - 3s 50ms/step - loss: 0.5026 - accuracy: 0.7565 - val_loss: 0.5483 - val_accuracy: 0.7460
Epoch 8/10
63

❓ **Question** ❓ Evaluate your model on the test set

In [14]:
# YOUR CODE HERE
# Evaluate the model on the test set

loss, accuracy = model.evaluate(X_test_pad, y_test)

print("Test loss: {:.2f}".format(loss))
print("Test accuracy: {:.2f}%".format(accuracy*100))


79/79 [==============================] - 1s 14ms/step - loss: 0.5827 - accuracy: 0.7176
Test loss: 0.58
Test accuracy: 71.76%


# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [15]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [17]:
# YOUR CODE HERE

# Load the pretrained word2vec model
word2vec_transfer = api.load("glove-wiki-gigaword-50")


In [21]:
# What is in word2vec_transfer?
print(type(word2vec_transfer))

# What is the dimension of the word vectors?
print(word2vec_transfer.vector_size)

# What is the vector for the word "computer"?
print(word2vec_transfer["computer"])


<class 'gensim.models.keyedvectors.KeyedVectors'>
50
[ 0.079084 -0.81504   1.7901    0.91653   0.10797  -0.55628  -0.84427
 -1.4951    0.13418   0.63627   0.35146   0.25813  -0.55029   0.51056
  0.37409   0.12092  -1.6166    0.83653   0.14202  -0.52348   0.73453
  0.12207  -0.49079   0.32533   0.45306  -1.585    -0.63848  -1.0053
  0.10454  -0.42984   3.181    -0.62187   0.16819  -1.0139    0.064058
  0.57844  -0.4556    0.73783   0.37203  -0.57722   0.66441   0.055129
  0.037891  1.3275    0.30991   0.50697   1.2357    0.1274   -0.11434
  0.20709 ]


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [24]:
# YOUR CODE HERE
# Size of the vocabulary
vocab_size = len(word2vec_transfer)

# Size of the embedding space
embedding_dim = word2vec_transfer.vector_size

print("Vocabulary size: ", vocab_size)
print("Embedding size: ", embedding_dim)


Vocabulary size:  400000
Embedding size:  50


❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [25]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)


❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [26]:
# YOUR CODE HERE
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='pre', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='pre', maxlen=200)


❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [27]:
# YOUR CODE HERE
# Re-initialize the model and fit it on the new embedded and padded training sentences with
# early stopping patience of 5 epochs and
# batch size of 32 and
# 10 epochs

model = Sequential([
    Masking(mask_value=0.0, input_shape=(200, 50)),
    LSTM(20, activation='tanh'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model using rmsprop optimizer and binary_crossentropy loss
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X_train_pad_2,
    y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    callbacks=[early_stopping],
)


Epoch 1/10
63/63 [==============================] - 8s 69ms/step - loss: 0.6877 - accuracy: 0.5600 - val_loss: 0.6791 - val_accuracy: 0.6080
Epoch 2/10
63/63 [==============================] - 3s 52ms/step - loss: 0.6642 - accuracy: 0.6140 - val_loss: 0.6459 - val_accuracy: 0.6340
Epoch 3/10
63/63 [==============================] - 3s 52ms/step - loss: 0.6232 - accuracy: 0.6740 - val_loss: 0.6186 - val_accuracy: 0.6680
Epoch 4/10
63/63 [==============================] - 3s 51ms/step - loss: 0.6129 - accuracy: 0.6660 - val_loss: 0.6034 - val_accuracy: 0.6780
Epoch 5/10
63/63 [==============================] - 3s 51ms/step - loss: 0.6018 - accuracy: 0.6915 - val_loss: 0.5982 - val_accuracy: 0.6800
Epoch 6/10
63/63 [==============================] - 3s 52ms/step - loss: 0.5895 - accuracy: 0.6945 - val_loss: 0.5925 - val_accuracy: 0.7000
Epoch 7/10
63/63 [==============================] - 3s 52ms/step - loss: 0.6004 - accuracy: 0.6870 - val_loss: 0.5988 - val_accuracy: 0.6760
Epoch 8/10
63

In [28]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')


The accuracy evaluated on the test set is of 70.080%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously